# this notebook is very simple xgboost
# rarely feature engineering 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
submission = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
columns_description = {
    'PassengerId': "'gggg_pp' Id, 'gggg': group id, 'pp': personal id within group",
    'HomePlanet': "Hometown planet",
    'CryoSleep': "frozen sleep",
    'Cabin': "cabin number where the passenger is staying", 
    'Destination': "planet destination",
    'Age': 'age', 
    'VIP': "paid for special VIP service",
    'RoomService': "billed at luxury amenity",
    'FoodCourt': "billed at luxury amenity",
    'ShoppingMall': "billed at luxury amenity",
    'Spa': "billed at luxury amenity",
    'VRDeck': "billed at luxury amenity",
    'Name': "name",
    'Transported': "Target"
}

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
train.shape, test.shape

## Null data check

In [ ]:
null_df = pd.DataFrame()
null_cnt = []
null_percent = []
for col in train.columns:
    cnt = train[col].isnull().sum()
    null_cnt += [cnt]
    null_percent += [cnt / len(train) * 100]
null_df['feature'] = train.columns
null_df['null_count'] = null_cnt
null_df['null_percent'] = null_percent
null_df.sort_values('null_count', ascending=False, inplace=True)
null_df

In [ ]:
import missingno as msno
msno.matrix(df=train.iloc[:, :], figsize=(8, 8))

In [ ]:
msno.matrix(df=test.iloc[:, :], figsize=(8, 8))

In [ ]:
feats_cat = []
feats_num = []
for col in train.columns:
    if col in ['PassengerId', 'Transported']:
        continue
    if train[col].dtype == 'object':
        feats_cat += [col]
    else:
        feats_num += [col]
        
feats_cat, feats_num

## Target label

In [ ]:
import matplotlib.pyplot as plt
f, ax = plt.subplots(figsize=(8, 8))
train['Transported'].value_counts().plot(ax=ax, kind='bar', ylabel='frequency')

### continuous & categorical feature

In [ ]:
num_feats, cat_feats = [], []
for col in train.columns:
    if train[col].dtype == 'object':
        cat_feats += [col]
    elif train[col].dtype == 'float64':
        num_feats += [col]

In [ ]:
num_feats, cat_feats

### PassengerId

In [ ]:
train['PassengerId'].isnull().sum()

### HomePlanet

In [ ]:
train['HomePlanet'].value_counts()

In [ ]:
def Target_True_func(x):
    return (x == True).sum()

def Target_False_func(x):
    return (x == False).sum()
    
train[['HomePlanet', 'Transported']].groupby('HomePlanet').agg([Target_True_func, Target_False_func])

In [ ]:
pd.crosstab(train['HomePlanet'], train['Transported'], margins=True)

In [ ]:
train[['HomePlanet', 'Transported']].groupby('HomePlanet').mean().sort_values('Transported', ascending=False).plot.bar()

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(1, 2)
sns.countplot(x='HomePlanet', data=train, ax=ax[0])
sns.countplot(x='HomePlanet', hue='Transported',data=train, ax=ax[1])
plt.show()

In [ ]:
cat_feats

### CryoSleep

In [ ]:
train['CryoSleep'].value_counts()

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(10, 6))
train[['CryoSleep', 'Transported']].groupby('CryoSleep').mean().plot.bar(ax=ax[0])
sns.countplot(x='CryoSleep', data=train, ax=ax[1], hue='Transported')
plt.show()

In [ ]:
sns.factorplot('HomePlanet', 'Transported', hue='CryoSleep', data=train)

In [ ]:
cat_feats

### Cabin

In [ ]:
train['Cabin'].value_counts()

In [ ]:
feat_extension = ['PassengerId', 'Cabin']

In [ ]:
cat_feats

### Destination

In [ ]:
train['Destination'].value_counts()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 6))
train[['Destination', 'Transported']].groupby('Destination').mean().plot.bar(ax=ax[0])
sns.countplot(x='Destination', hue='Transported', data=train, ax=ax[1])

In [ ]:
pd.crosstab(index=train['Destination'], columns=train['HomePlanet'], normalize='index', margins=True)

### VIP

In [ ]:
train['VIP'].value_counts()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 6))
train[['VIP', 'Transported']].groupby('VIP').mean().plot.bar(ax=ax[0])
sns.countplot(x='VIP', hue='Transported', data=train, ax=ax[1])

In [ ]:
cat_feats

### Name

In [ ]:
len(train['Name'].unique()), len(train)

In [ ]:
cat_feats, num_feats

### Age

In [ ]:
train['Age'].max(), train['Age'].min(), train['Age'].mean()

In [ ]:
sns.kdeplot(x='Age', hue='Transported', data=train)

### RoomService, FoodCourt, ShoppingMall, Spa, VRDeck

In [ ]:
import copy
num_feats_ = copy.deepcopy(num_feats)
num_feats_.remove('Age')
fig, axes = plt.subplots(len(num_feats_), 1, figsize=(6, 15))
for i, col in enumerate(num_feats_):
    print(train[col].max(), train[col].min(), train[col].mean())
    sns.kdeplot(x=col, hue='Transported', data=train, ax=axes[i])

In [ ]:
g = sns.pairplot(train[num_feats_+['Transported']], hue='Transported', kind='kde')
g.set(ylim=(0, 5000), xlim=(0, 5000))

## Feature engineering

In [ ]:
dataset = pd.concat([train, test], axis=0).reset_index(drop=True)
dataset

In [ ]:
dataset = dataset.drop(columns=['PassengerId', 'Cabin', 'Name'])

In [ ]:
dataset.head()

In [ ]:
cat_feats

In [ ]:
cat_feats = list(set(cat_feats) - set(['PassengerId', 'Cabin', 'Name']))
cat_feats

In [ ]:
dataset = pd.get_dummies(dataset, columns=cat_feats)
dataset.head()

## model (XGBoost)

In [ ]:
train_length = len(train)
train, test = dataset[:train_length], dataset[train_length:]

In [ ]:
train_y = train['Transported']
train_x = train.drop('Transported', axis=1)

In [ ]:
test = test.drop('Transported', axis=1)

In [ ]:
from xgboost import plot_importance
from xgboost import XGBClassifier

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(train_x, train_y, random_state=0, shuffle=True, stratify=train_y, test_size=0.2)

In [ ]:
x_train.shape, x_val.shape, y_train.shape

In [ ]:
from sklearn.metrics import accuracy_score
xgb = XGBClassifier()
xgb.fit(x_train, y_train, eval_set=[(x_val, y_val)], eval_metric='error', verbose=True)

In [ ]:
plot_importance(xgb)

In [ ]:
submission['Transported'] = xgb.predict(test).astype('bool')
submission.to_csv('sampleSubmission.csv', index=False)

## test score: 0.78138